## Attributes Classificator

In this notebook I will try to classify all the attributes present in a face.
This could be useful to use the attributes as condition in input of conditional models

In [1]:
from classes.Architectures import VGGNetLike
import numpy as np
import pandas as pd
from os.path import join as opj

import os

import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback
from imutils import paths
from tensorflow.data import AUTOTUNE
import matplotlib.pyplot as plt


wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


True

In [89]:
images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"
anno_dir=r"/home/matteo/NeuroGEN/Dataset/Anno/list_attr_celeba.csv"
#other important definitions

EPOCHS=50
BS=64
INIT_LR=1e-4

config={}
config["epochs"]=EPOCHS
config["BS"]=BS
config["init_lr"]=INIT_LR

wandb.init(project="TorVergataExperiment-Generative",config=config,name="CelebA_Attribute_Prediciton")

wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


## Load data and indices

In [6]:
df=pd.read_csv(anno_dir,sep=",")

In [72]:
@tf.function
def load_images(imagePath,attr):

    #attr=df.iloc[idx].drop("image_id",axis=1).values
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128)) / 255.0

    #eventually load other information like attributes here
    
    # return the image and the extra info
    
    
    return image,attr

In [70]:
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

train_idxs=[int(i.split("/")[-1].split(".jpg")[0])-1 for i in train_imgs]
val_idxs=[int(i.split("/")[-1].split(".jpg")[0])-1 for i in val_imgs]
test_idxs=[int(i.split("/")[-1].split(".jpg")[0])-1 for i in test_imgs]

train_attr=[df.iloc[idx].drop("image_id").values for idx in train_idxs]
val_attr=[df.iloc[idx].drop("image_id").values for idx in val_idxs]
test_attr=[df.iloc[idx].drop("image_id").values for idx in test_idxs]


print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")

[INFO] loading image paths...
[TRAINING]	 138545
[VALIDATION]	 17318
[TEST]		 17319


In [79]:
train_attr=np.array(train_attr).astype("int")
val_attr=np.array(val_attr).astype("int")
test_attr=np.array(test_attr).astype("int")

In [85]:
#TRAINING 

train_dataset = tf.data.Dataset.from_tensor_slices((train_imgs,train_attr))
train_dataset = (train_dataset
    .shuffle(1024)
    .map(load_images)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices((val_imgs,val_attr))
val_dataset = (val_dataset
    .shuffle(1024)
    .map(load_images)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices((test_imgs,test_attr))
test_dataset = (test_dataset
    .shuffle(1024)
    .map(lambda x,y:load_images(x,y), num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [86]:
for x,y in train_dataset:
    print(x.shape)
    print(y.shape)
    break

(64, 128, 128, 3)
(64, 40)


## Define the model

In [95]:
model=VGGNetLike((128,128,3),40)


opt=tf.keras.optimizers.Adam(INIT_LR)
callbacks=[WandbCallback()]

model.compile(loss="categorical_crossentropy",optimizer=opt,metrics="accuracy")


In [6]:
model.summary()

Model: "VGGNetLike_Classificator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 activation (Activation)     (None, 128, 128, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 activation_1 (Activation)   (None, 128, 128, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                          

## Train the model

In [ ]:
model.fit(train_dataset,epochs=EPOCHS,steps_per_epoch=ts,callbacks=callbacks)

Epoch 1/50
2164/2164 [==============================] - 630s 290ms/step - loss: -338.8349 - accuracy: 0.0062
Epoch 2/50
2164/2164 [==============================] - 602s 278ms/step - loss: -370.3544 - accuracy: 0.0037
Epoch 3/50
2164/2164 [==============================] - 622s 288ms/step - loss: -373.7160 - accuracy: 0.0054
Epoch 4/50
2164/2164 [==============================] - 620s 287ms/step - loss: -373.6825 - accuracy: 0.0053
Epoch 5/50
2164/2164 [==============================] - 625s 289ms/step - loss: -373.6770 - accuracy: 0.0053
Epoch 6/50
2164/2164 [==============================] - 624s 289ms/step - loss: -373.7118 - accuracy: 0.0054
Epoch 7/50
2164/2164 [==============================] - 622s 288ms/step - loss: -373.7362 - accuracy: 0.0053
Epoch 8/50
2164/2164 [==============================] - 623s 288ms/step - loss: -373.7488 - accuracy: 0.0053
Epoch 9/50
2164/2164 [==============================] - 621s 287ms/step - loss: -373.7485 - accuracy: 0.0053
Epoch 10/50
1511/21